In [1]:
import json
import pandas as pd
import numpy as np
import sys
import glob

sys.path.insert(0, '../includes/')
import helper
import plot_config

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Band, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [25]:
params = {
    'report_files': '/home/bhargavjoshi/RL-mc-data/test-dir/openaigym.episode_batch.0.*.stats.json',
    'episodes': 160,
}


def parse_reward_file(fn):
    with open(fn, 'r') as f:
        report = json.load(f)

    assert len(report['episode_rewards']) >= params['episodes'], 'too few episodes recorded'
    return report['episode_rewards']


def load_rewards(report_files):
    fns = glob.glob(report_files)
    if not fns: raise FileNotFoundError('no report files found')

    rewards = []
    for i, fn in enumerate(fns):
        rewards.append(parse_reward_file(fn)[:params['episodes']])

    return np.mean(rewards, axis=0), np.std(rewards, axis=0)

mean_rewards_1, std_rewards_1 = load_rewards(params['report_files'])
episode_1 = np.array(range(1, params['episodes']+1))

In [38]:
source1 = ColumnDataSource(dict(
    x = episode_1,
    mean = mean_rewards_1,
    upper = mean_rewards_1 + std_rewards_1,
    lower = mean_rewards_1 - std_rewards_1
))

p1 = figure(
    title = "TD learning in SNN with rate neurons: MountainCar (gym + NEST toolchain)",
    x_axis_label = "Episodes",
    y_axis_label = "Mean Rewards"
)

band1 = Band(base='x', lower='lower', upper='upper', source=source1, level='underlay',
            fill_alpha=0.2, line_width=1, line_color='black', fill_color='blue')
p1.line(x=episode_1, y=mean_rewards_1, line_width=2, color='blue')
p1.add_layout(band1)

In [39]:
show(p1)

In [43]:
# save data in csv file
df = pd.DataFrame({'mean': mean_rewards_1, 'std': std_rewards_1})
df.to_csv('test-toolchain-mc.csv', index=0)